<a href="https://colab.research.google.com/github/takakishi/HEC_DS_ML_project/blob/main/src/logistic_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup and Import Data

In [20]:
!pip install spacy
!python -m spacy download fr_core_news_sm
!pip install textstat

2023-12-20 07:54:40.920749: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-20 07:54:40.920814: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-20 07:54:40.922209: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-20 07:54:42.150498: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 28.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('fr_core_news_sm')


In [21]:
# Libraries
import pandas as pd
import numpy as np
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import spacy
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stopwords
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from scipy.sparse import hstack
import textstat
from sklearn.impute import SimpleImputer
from sklearn.decomposition import TruncatedSVD

In [22]:
# Import Data
sample_submission = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/sample_submission.csv')
training_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/training_data.csv')
unlabelled_test_data = pd.read_csv('https://raw.githubusercontent.com/takakishi/HEC_DS_ML_project/main/data/data_raw/unlabelled_test_data.csv')

In [23]:
sample_submission.head

<bound method NDFrame.head of         id difficulty
0        0         A1
1        1         A1
2        2         A1
3        3         A1
4        4         A1
...    ...        ...
1195  1195         A1
1196  1196         A1
1197  1197         A1
1198  1198         A1
1199  1199         A1

[1200 rows x 2 columns]>

In [24]:
training_data.head

<bound method NDFrame.head of         id                                           sentence difficulty
0        0  Les coûts kilométriques réels peuvent diverger...         C1
1        1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2        2  Le test de niveau en français est sur le site ...         A1
3        3           Est-ce que ton mari est aussi de Boston?         A1
4        4  Dans les écoles de commerce, dans les couloirs...         B1
...    ...                                                ...        ...
4795  4795  C'est pourquoi, il décida de remplacer les hab...         B2
4796  4796  Il avait une de ces pâleurs splendides qui don...         C1
4797  4797  Et le premier samedi de chaque mois, venez ren...         A2
4798  4798  Les coûts liés à la journalisation n'étant pas...         C2
4799  4799  Sur le sable, la mer haletait de toute la resp...         C2

[4800 rows x 3 columns]>

In [25]:
unlabelled_test_data.head

<bound method NDFrame.head of         id                                           sentence
0        0  Nous dûmes nous excuser des propos que nous eû...
1        1  Vous ne pouvez pas savoir le plaisir que j'ai ...
2        2  Et, paradoxalement, boire froid n'est pas la b...
3        3  Ce n'est pas étonnant, car c'est une saison my...
4        4  Le corps de Golo lui-même, d'une essence aussi...
...    ...                                                ...
1195  1195  C'est un phénomène qui trouve une accélération...
1196  1196  Je vais parler au serveur et voir si on peut d...
1197  1197  Il n'était pas comme tant de gens qui par pare...
1198  1198      Ils deviennent dangereux pour notre économie.
1199  1199  Son succès a généré beaucoup de réactions néga...

[1200 rows x 2 columns]>

# Functions and Data Splitting

In [26]:
def preprocess_basic(text):
    text = text.lower()  # Convert to lower case
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    return text

training_data['processed_sentence_basic'] = training_data['sentence'].apply(preprocess_basic)

tfidf_vectorizer = TfidfVectorizer()
X_basic = tfidf_vectorizer.fit_transform(training_data['processed_sentence_basic'])
y_basic = training_data['difficulty']

X_train_basic, X_val_basic, y_train_basic, y_val_basic = train_test_split(X_basic, y_basic, test_size=0.2, random_state=42)

# Logit (with Hyperparameter Tuning), etc

In [27]:
training_data['sentence_length'] = training_data['sentence'].apply(lambda x: len(x.split()))
scaler = StandardScaler()
length_scaled = scaler.fit_transform(training_data[['sentence_length']])

X_with_length_basic = hstack((X_basic, length_scaled))

X_train_length_basic, X_val_length_basic, y_train_length_basic, y_val_length_basic = train_test_split(X_with_length_basic, y_basic, test_size=0.2, random_state=42)

log_reg_length_basic = LogisticRegression(random_state=42, max_iter=1000)
log_reg_length_basic.fit(X_train_length_basic, y_train_length_basic)

y_val_pred_length_basic = log_reg_length_basic.predict(X_val_length_basic)
accuracy_length_basic = accuracy_score(y_val_length_basic, y_val_pred_length_basic)

average_type = 'macro'  # can be 'micro' or 'weighted'

precision_length_basic, recall_length_basic, f1_length_basic, _ = precision_recall_fscore_support(
    y_val_length_basic, y_val_pred_length_basic, average=average_type
)

print("Accuracy with Sentence Length Feature for Basic Preprocessing:", accuracy_length_basic)
print("Precision with Sentence Length Feature for Basic Preprocessing:", precision_length_basic)
print("Recall with Sentence Length Feature for Basic Preprocessing:", recall_length_basic)
print("F1-Score with Sentence Length Feature for Basic Preprocessing:", f1_length_basic)

Accuracy with Sentence Length Feature for Basic Preprocessing: 0.475
Precision with Sentence Length Feature for Basic Preprocessing: 0.47060310254768645
Recall with Sentence Length Feature for Basic Preprocessing: 0.4726081395493377
F1-Score with Sentence Length Feature for Basic Preprocessing: 0.4647067045596908


In [28]:
unlabelled_test_data['processed_sentence'] = unlabelled_test_data['sentence'].apply(preprocess_basic)
unlabelled_test_data['sentence_length'] = unlabelled_test_data['sentence'].apply(lambda x: len(x.split()))
X_unlabelled_tfidf = tfidf_vectorizer.transform(unlabelled_test_data['processed_sentence'])
length_scaled_unlabelled = scaler.transform(unlabelled_test_data[['sentence_length']])

X_unlabelled_with_length = hstack((X_unlabelled_tfidf, length_scaled_unlabelled))

predictions = log_reg_length_basic.predict(X_unlabelled_with_length)

submission = pd.DataFrame({
    'id': unlabelled_test_data['id'],
    'difficulty': predictions
})

In [29]:
'''
from google.colab import drive
drive.mount('/content/drive')

import joblib

joblib.dump(log_reg_length_basic, '/content/drive/My Drive/Colab Notebooks/log_reg_length_basic.joblib')
joblib.dump(log_reg_length_basic, '/content/drive/My Drive/Colab Notebooks/log_reg_length_basic.joblib')
joblib.dump(tfidf_vectorizer, '/content/drive/My Drive/Colab Notebooks/tfidf_vectorizer.joblib')
joblib.dump(scaler, '/content/drive/My Drive/Colab Notebooks/length_scaler.joblib')
'''

"\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nimport joblib\n\njoblib.dump(log_reg_length_basic, '/content/drive/My Drive/Colab Notebooks/log_reg_length_basic.joblib')\njoblib.dump(log_reg_length_basic, '/content/drive/My Drive/Colab Notebooks/log_reg_length_basic.joblib')\njoblib.dump(tfidf_vectorizer, '/content/drive/My Drive/Colab Notebooks/tfidf_vectorizer.joblib')\njoblib.dump(scaler, '/content/drive/My Drive/Colab Notebooks/length_scaler.joblib')\n"

In [30]:
# from google.colab import files
# files.download('length_scaler.pkl')

In [31]:
# submission.set_index('id', inplace=True)

In [32]:
# submission

In [33]:
# unlabelled_test_data['difficulty'] = list(map(lambda x: "A1", unlabelled_test_data['sentence'].tolist()))
# predictions = y_val_pred_length_basic.drop(columns=['sentence'], inplace = False)
# predictions.set_index('id',inplace = True)

'''
predictions = list(y_val_pred_length_basic.tolist())
predictions = pd.DataFrame(predictions)

# Creating an 'id' column that is a range from 0 to length of DataFrame - 1
predictions['id'] = range(len(predictions))

# Set the 'id' column as index
predictions.set_index('id', inplace=True)
predictions

submission.to_csv('submission.csv')
'''

"\npredictions = list(y_val_pred_length_basic.tolist())\npredictions = pd.DataFrame(predictions)\n\n# Creating an 'id' column that is a range from 0 to length of DataFrame - 1\npredictions['id'] = range(len(predictions))\n\n# Set the 'id' column as index\npredictions.set_index('id', inplace=True)\npredictions\n\nsubmission.to_csv('submission.csv')\n"

In [34]:
submission.head

<bound method NDFrame.head of         id difficulty
0        0         C2
1        1         A2
2        2         A1
3        3         A1
4        4         C2
...    ...        ...
1195  1195         B1
1196  1196         A2
1197  1197         C2
1198  1198         A1
1199  1199         A2

[1200 rows x 2 columns]>

In [35]:
# from google.colab import drive
# drive.mount('/content/drive', force_remount=True)
# install Kaggle
# ! pip install kaggle
# !mkdir ~/.kaggle
# !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [36]:
# ! kaggle competitions submit -c detecting-french-texts-difficulty-level-2023 -f submission.csv -m "Sample submission"